# TIME SHIFTING

When studying the correlations between the indicators and the GDP, we should take into consideration that the effects of said indicators might not be immediate, but built up over the course of the years. Thus, in this notebook we will be exploring the correlations after applying time shifting, that is, comparing the values of indicators for year X to the GDP of year X + N.

Note that we could also do the opposite: study the effect of GDP on the indicators. In order to have a full view of these interactions, we can choose to show this on the last table of the notebook by selecting negative values in the range of shifts.

## IMPORTS AND CONSTANTS

In [ ]:
import os
import numpy as np
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

import warnings
warnings.filterwarnings("ignore")

from Project.Utils.shift_corr import shift_corr
from Project.Utils.max_corr import max_corr

PVALUE_VAR = 0.05

read_path = os.getcwd() + '/Output/'

col_country = 'Country'
col_region = 'Region'
col_year = 'Year'
col_gdp = 'GDP'
col_shift = 'Shift'

## LOADING DATAFRAME

We will load the ready-to-use GoldDataframe, which will be the base for our study in this notebook.

In [ ]:
# Read and display the GoldDataframe
df = pd.read_csv(read_path + 'GoldDataframe.csv', index_col = [col_country, col_region, col_year])
df

## COMPUTING CORRELATIONS

In order to be able to show the results in a more agile way, we pre-compute them in the following cell. The tables for each view mode will be stored in the show_dictionary, so the display cell only has to change which table is shown and what conditions use to filter it, instead of processing it once again.

The range for the shifts that can be shown is defined here, as well.

In [ ]:
# The show_dict will be used to store the full tables that can be seen for each view mode, using said mode as the key.
show_list = [col_country, col_region]
show_dict = {}
for method in show_list:
    shift_corr_df = shift_corr(df, method)
    show_dict[method] = shift_corr_df

# After computing all the modes, the shifts range is established by the minimum and maximum ranges for all the tables.
shifts_range = range(
    min(min(show_dict[how].index.get_level_values(col_shift)) for how in show_list),
    max(max(show_dict[how].index.get_level_values(col_shift)) for how in show_list)
)

## DISPLAYING RESULTS

The following cell shows the strongest correlation between the indicator and the GDP, for the given area selected. Hovering over the cell, it will be displayed what shift that correlation corresponds to.

By default, only positive shifts are shown, which translates into a shifted GDP, that is, the effects of indicators over future GDP. Negative values can be selected too, which will show the effects of the GDP over indicators' future values.

In [ ]:
def table_MaxCorr(selection: str, sh_range: tuple):    

    # Load the corresponding Dataframe and apply the user-introduced restrictions.
    df = show_dict[selection]
    min_sh = sh_range[0]
    max_sh = sh_range[1]
    df = df.loc[(min_sh <= df.index.get_level_values(col_shift)) & (df.index.get_level_values(col_shift) <= max_sh)]
    max_corr_df, max_corr_index_df = max_corr(df, selection, PVALUE_VAR, raw = False)

    # Apply style and display.
    df_s = max_corr_df.style
    df_s.set_tooltips(max_corr_index_df.applymap(lambda x:
                                                    'No data' if np.isnan(x) else 
                                                    'Shift: ' + str(int(x))
                                                    
                                                )
                    )
    df_s.background_gradient(cmap='RdBu')

    display(df_s)

In [ ]:
dropdown = widgets.Dropdown(
    options = show_list,
    description = 'Show: '
)

intslider = widgets.IntRangeSlider(
    value = (max(0, shifts_range.start), shifts_range.stop),
    min = shifts_range.start,
    max = shifts_range.stop,
    step = shifts_range.step,
    description = 'Shifts Range: ')

widgets.interact(table_MaxCorr, selection = dropdown, sh_range = intslider)